In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import h5py
from pprint import pprint
import numpy as np
import logging 
logging.getLogger('matplotlib').setLevel(logging.WARNING)
import matplotlib.pyplot as plt
from scipy.special import entr
from drift_ml.datasets.bosch_cnc_machining.utils.utils import augment_xyz_samples
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import NPYBoschCNCDataLoader


In [12]:
base_loader = NPYBoschCNCDataLoader(metadata_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/metadata_ws4096.pkl")
base_loader.load_data(
    sample_data_x_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_x_raw_ws4096.npy",
    sample_data_y_path="/home/tbiegel/nico_files/drift_ml/src/drift_ml/datasets/bosch_cnc_machining/extracted_features/npy/sample_data_y_raw_ws4096.npy"
    )

In [22]:
sudden_config = {
    "base_config": {
        "train_size": 0.5,
        "val_size": 0.0,
        "test_size": 0.5,
        "machines": None,
        "processes": None,
        "periods": None,
    },
    "drift_config": [
        {
            "length": 10000,
            "type": "constant",
            "only_test": False,
            "machines": None,
            "processes": None,
            "periods": ["Feb_2019", "Aug_2019"],
            "transform_fn": None,
        },
        {
            "length": 10000,
            "type": "constant",
            "only_test": False,
            "machines": None,
            "processes": None,
            "periods": None,
            "transform_fn": lambda x: augment_xyz_samples(x, pitch_deg=30),
        },
    ],
}

In [20]:
from drift_ml.datasets.bosch_cnc_machining.utils.dataloader import DriftDataLoader

In [21]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=sudden_config)
samples, labels = drift_dataloader.access_test_drift_samples_stft(index=999, length=10000)

print(samples.shape)
print(labels.shape)

taking 9000 samples from config 0, indices 999 to 9999
taking 999 samples from config 1, indices 0 to 999
(9999, 3, 129, 33)
(9999,)


In [23]:
from tsfresh import feature_extraction
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=sudden_config)
samples, labels = drift_dataloader.access_base_samples_tsfresh(featureset=feature_extraction.MinimalFCParameters(), dataset="test")

print(samples.shape)
print(labels.shape)

Feature Extraction:  15%|█▍        | 943/6377 [00:09<00:10, 508.74it/s]

In [8]:
incremental_config = {
    "base_config": {
        "train_size": 0.3,
        "val_size": 0.2,
        "test_size": 0.5,
        "machines": None,
        "processes": None,
        "periods": None,
    },
    "drift_config": [
        {
            "start": 0,
            "end": 10000,
            "type": "linear",
            "part_1": {
                "only_test": False,
                "machines": None,
                "processes": None,
                "periods": None,
                "transform_fn": None,
            },
            "part_2": {
                "only_test": False,
                "machines": None,
                "processes": None,
                "periods": None,
                "transform_fn": lambda x: augment_xyz_samples(x, pitch_deg=30),
            },
        }
    ],
}


In [14]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=incremental_config)
samples = drift_dataloader.access_test_drift_samples_stft(index=0, length=10200)

print(samples.shape)

taking 10000 samples from config 0, indices 0 to 10000
(10000, 3, 129, 33)


In [38]:
import pandas as pd
from tsfresh import feature_extraction

features = pd.read_pickle(
    "/home/tbiegel/nico_files/bosch_cnc_machining/features_and_data/all_top_30_features.pkl"
)
feature_settings = feature_extraction.settings.from_columns(features)

In [19]:
drift_dataloader = DriftDataLoader(baseloader=base_loader, config=incremental_config)
samples = drift_dataloader.access_test_drift_samples_tsfresh(featureset=feature_extraction.MinimalFCParameters(), index=0, length=1000)
print(samples.shape)

taking 1000 samples from config 0, indices 0 to 1000


Feature Extraction: 100%|██████████| 300/300 [00:00<00:00, 404.89it/s]

(1000, 30)
